In [ ]:
import sounddevice as sd
import numpy as np
import librosa
import joblib

# Modeli yükleyin
model = joblib.load('..\Model\VoiceModel.pkl')

# Sınıf etiketlerini manuel olarak tanımlayın
class_labels = {0: 'elif', 1: 'irem', 2: 'nazlı'}

# Özellik adlarını yükle
feature_names = np.load('..\Model\feature_names.npy', allow_pickle=True)
print(f"Modelin beklediği özellikler: {feature_names}")

# Ses kaydı parametreleri
SAMPLE_RATE = 22050  # Ses örnekleme hızı
DURATION = 3  # Ses kaydının süresi (saniye cinsinden)
HOP_LENGTH = 512  # Librosa için hop_length

# Ses kaydını alma fonksiyonu
def record_audio(duration, sample_rate):
    print("Ses kaydediliyor...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Kaydın bitmesini bekle
    print("Ses kaydı tamamlandı.")
    return audio.flatten()  # Ses verisini 1D dizisine dönüştür

# MFCC, Chroma ve diğer özellikleri çıkartma
def extract_features_from_audio(y, sr):
    # MFCC Özellikleri
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40, hop_length=HOP_LENGTH)
    mfcc_mean = np.mean(mfcc, axis=1)  # MFCC'nin ortalamaları
    mfcc_std = np.std(mfcc, axis=1)    # MFCC'nin standart sapmaları

    # Chroma Özellikleri
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=HOP_LENGTH)
    chroma_mean = np.mean(chroma, axis=1)

    # Spectral Özellikler
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=HOP_LENGTH)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr, hop_length=HOP_LENGTH)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, hop_length=HOP_LENGTH)

    # ZCR (Zero-Crossing Rate) ve RMS (Root Mean Square)
    zcr = librosa.feature.zero_crossing_rate(y, hop_length=HOP_LENGTH)
    rms = librosa.feature.rms(y=y)

    # Özellikleri sırasıyla birleştirme (modelin beklediği sıraya göre)
    features = np.hstack([
        mfcc_mean[0:24],  # İlk 24 MFCC özelliği
        mfcc_std[0:24],   # İlk 24 MFCC standart sapması
        chroma_mean[0:5],  # İlk 5 Chroma özelliği
        np.mean(rms),  # RMS özelliği
    ])

    # 58 özellik bekleniyor, bu eksiklik varsa burada tamamlayabilirsiniz
    if len(features) < len(feature_names):
        # Eksik özellikleri sıfırla tamamlayabiliriz (veya uygun varsayılan değerlerle)
        features = np.hstack([features, np.zeros(len(feature_names) - len(features))])
    
    return features

# Ses kaydını al ve özellikleri çıkart
audio_data = record_audio(DURATION, SAMPLE_RATE)

# Özellikleri çıkart
features = extract_features_from_audio(audio_data, SAMPLE_RATE)

# Özellik sayısını kontrol et (58 özellik)
print(f"Özellik sayısı: {len(features)}")

# Tahmin yap (scaler kullanmadan)
prediction = model.predict([features])

# Tahmini sınıf adını al
predicted_class = class_labels[prediction[0]]

print(f"Bu konuşma {predicted_class} tarafından yapılmıştır.")


OSError: [Errno 22] Invalid argument: '..\\Model\x0cfeature_names.npy'